In [ ]:
import math
import numpy as np
import pandas as pd
from typing import Tuple, List, Optional, Generator

# MA7010 Assignment 2

In [ ]:
def sign(n: int):
    return 1 if n > 0 else -1

def gcd_recursion(a: int, b: int) -> int:
    """ Find GCD by recursing on a mod b """
    if(b == 0):
        return sign(a) * a

    return gcd_recursion(b, a % b)

def gcd_euclidian(a: int, b: int) -> int:
    """ Euler algo """

    # a = q * b - r
    q = a // b
    r = a - q * b
    
    # Stopping condition: r = 0
    if r == 0:
        return b
    
    # next iteration b = r * q_2 + r
    return gcd_euclidian(b, r)

def gcd(a: int, b: int, fn = None) -> int:
    """  """
    # ensure a >= b
    a, b = (a, b) if a >= b else (b, a)

    fn = fn or gcd_euclidian
    return fn(a, b)

def extended_euclidian(a: int, b: int, state:Tuple[int, int, int] = None) -> int:
    """ Euler algo: find gcd(a, b) and x & y s.t. gcd(a, b) = x * a - y * b """

    # First time through, setup state, and ensure 'a' > 'b'
    state =  state or (((a, 1, 0), (b, 0, 1)) if a >= b else ((b, 1, 0), (a, 0, 1)))
   
    u = state[0]
    v = state[1]
    
    # Stopping condition: r = 0
    if v[0] <= 0:
        gcd = u[0]
        x = u[1] if a >= b else u[2]
        y = u[2] if a >= b else u[1]
        return (gcd, x, y)
    
    # a = q * b - r
    q = u[0] // v[0]
    w =  tuple(ui - q * vi for ui, vi in zip(u, v))
    state = (v, w)
    
    # next iteration b = r * q_2 + r
    return extended_euclidian(a, b, state)

def modulo_inverse(a, n):
    if a < 0:
        gcd, x, y = extended_euclidian(a + n, n)
    else:
        gcd, x, y = extended_euclidian(a, n)
    if gcd == 1:
        assert (a * x) % n == 1
        return x if x >= 0 else n + x
    return np.NaN
    
def sieve_of_eratosthenes(n: int) -> Tuple[int,...]:
    """ Simple sieve to find primes up to n """
    n = n if n > 0 else -n
    if n < 2:
        return tuple()

    is_prime_sieve = np.full(n + 1, True)
    is_prime_sieve[0:2] = False
    is_prime_sieve[4::2] = False
    
    # Start with 2 and odd numbers from 3 to n
    sqrt_n = math.ceil(np.sqrt(n))
    for si in range(3, sqrt_n + 1, 2):
        if is_prime_sieve[si]:
            # Mark every multiple of si from si^2
            is_prime_sieve[si ** 2::si] = False
    return tuple(int(i) for i in np.flatnonzero(is_prime_sieve))

def primes_below(n: int) -> Generator[int, int, None]:
    """ """
    for p in sieve_of_eratosthenes(n - 1):
        yield p
    return

def next_prime(n: int):
    """ """
    for p in primes_below((math.trunc(np.sqrt(n)) + 1) ** 2):
        if p > n:
            return p
    return np.NaN

def is_prime(n: int) -> bool:
    """ check if is a prime by division: O(log(sqrt(n))) """
    for i in range(2, math.trunc(np.sqrt(n)) + 1):
        if n % i == 0:
            return  False
    return True

def ifactors(n: int) -> Generator[Tuple[int, int], int, None]:
    """   """
    n_abs = sign(n) * n
    half_n = n_abs // 2
    is_prime = True
    for p in primes_below(half_n + 1):
        if p > half_n:
            break
        
        
        if n_abs % p == 0:
            is_prime = False
            k = 1
            m = n_abs // p
            while m % p == 0:
                m = m // p
                k += 1
            yield (p, k)
    
    if is_prime:
        yield (n_abs, 1)
                
    return

def divisors(n: int) -> Tuple[int, ...]:
    """ """
    factors = tuple(ifactors(n))
    factor_primes = np.array([f[0] for f in factors] + [0,])
    # add one to every power, as we generate powers as (i mod factor_powers[j])
    factor_powers = np.array([f[1] for f in factors] + [0,]) + 1
    
    factors_count = len(factors)
    divisors_count = np.prod(factor_powers)

    # calc product of array of each prime factor to some power, varying from 0 to the max given from ifactors fn
    ds = sorted(int(np.prod([factor_primes[j] ** (i // np.prod(factor_powers[j - factors_count:]) % factor_powers[j])
                             for j in range(factors_count)]))
                for i in range(divisors_count))
    return tuple(ds)

def co_primes(n: int):
    """ """
    return set([a for a in range(1, n) if gcd(a, n) == 1])

def totient(n: int) -> int:
    """ Euler's phi function is the number of values less than a that are co-prime with n """
    return len(co_primes(n))

def order_of_powers(g: int, n: int) -> List[int]:
    """ g ^ k % n for k being co-prime with phi(n) """

    # order_of_powers = sorted(set([pow(g, k, n) for k in co_primes(totient(n))]))
    # keep all calcs mod n to remove overflow errors
    ks = co_primes(totient(n))
    order_of_powers = set()
    g_k = 1
    for k in range(1, n):
        g_k = g_k * g  % n
        if k in ks:
            order_of_powers.add(g_k)
    return sorted(order_of_powers)

def order(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN

    a_k = 1
    for k in range(1, n):
        a_k = a_k * a  % n
        if a_k == 1:
            return k
    
    return np.NaN

def is_order_n(a: int, n: int):
    """ Multiplicative order of a mod n is the smallest k for which a^k mod n is 1 """
    if a > n or gcd(a, n) != 1:
        return np.NaN
    
    ord_n = totient(n)
    # we can do better than all k < n by only looking at divisors of totient(n)
    phi_n_divisors = divisors(ord_n)
    for k in phi_n_divisors:
        if pow(a, k, n) == 1:
            return k == ord_n
    
    return np.NaN

def cyclic_group(a, n, op):
    group = set([pow(a, k, n) for k in range(1, n)])
    return group

def primitive_roots(n):
    # g is a primitive root modulo n if for every integer a coprime to n, the ord_n(a) = phi(n) there is some integer k for which gk ≡ a (mod n)
    
    # check n is form 2, 4, p^s, 2p^s, where s is any positive integer and p is an odd prime
    #factors = [f for f in ifactors(n)]
    #if any((len(factors) < 1 or 2 < len(factors),  
    #        (len(factors) == 2 and (factors[0][0] != 2 or factors[0][1] > 1)),
    #        (len(factors) == 1 and factors[0][0] == 2 and factors[0][1] > 2),
    #        (len(factors) == 1 and factors[0][0] < 2))):
    #    return [] # Exception("No primitive roots exist")
    
    # find smallest primitive root
    phi_n = totient(n)
    gs = [a for a in co_primes(n) if order(a, n) == phi_n]

    if not gs:
        return []
    # There are phi(phi(n)) roots: return all roots using factors co-prime with phi(n)
    #prime_roots = order_of_powers(g, n)
     
    assert len(gs) == totient(phi_n)
    assert all(pow(g, phi_n, n) == 1 for g in gs)
    assert set(order_of_powers(gs[-1], n)) == set(gs)
    return gs

__Question 1  (10 marks)__

Consider all the numbers n in your range. Divide the set into two subsets:

A – the subset consisting of all n where there is at least one primitive root modulo n;

B – the subset consisting of all n where no primitive roots exist modulo n.

In [ ]:
lower_range = 800
upper_range = 950

subset_a = set(n for n in range(lower_range, upper_range + 1) if len(primitive_roots(n)) > 0)
subset_b = set(n for n in range(lower_range, upper_range + 1)) - subset_a

print(f"# with primitive roots: {len(subset_a)},\t# primes with primitive roots: {sum(1 for n in subset_a if is_prime(n))}")
print(f"# w/o  primitive roots: {len(subset_b)},\t# primes w/o primitive roots: {sum(1 for n in subset_b if is_prime(n))}")
print(sorted(subset_a))

__Question 2  (12 marks)__

a)	Explain why we can always find a primitive root modulo $p$ when $p$ is a prime.

A primitive root, modulo some number

b)	Express the number of primitive roots that exist modulo $p$ using the Euler Totient function and show that your answer correctly predicts the number of primitive roots for all primes in your given range.

In [ ]:
for i, n in enumerate([a for a in sorted(subset_a) if is_prime(a)]):
    print(f"{i+1}.\tn={n}\t# of primitive roots: {len(primitive_roots(n))}\tphi(phi(n)): {totient(totient(n))}")

c)	For the same range as Question 1 use the command ifactors in Maple to find the set C whose elements consist of numbers of the form $p^k (k > 1)$ or $2p^k (k > 1)$.

In [ ]:
def isform_pk_or_2pk(n):
    factors = [f for f in ifactors(n)]
    return ((len(factors) == 2 and factors[0][0] == 2 and factors[0][1] == 1 and factors[1][1] >= 1)
            or (len(factors) == 1 and factors[0][0] > 2 and factors[0][1] >= 1))

subset_c = set(n for n in range(lower_range, upper_range + 1) if isform_pk_or_2pk(n))
print(sorted(subset_c))
print(subset_a == subset_c)

d)	Hence form a conjecture about when primitive roots do and don’t exist.

__Question 3 (15 marks)__

Suppose n has the form n = pq where p and q are different primes both > 2.

a)	What is $\phi(n)$ in terms of $p$ and $q$?

b)	Suppose a is relatively prime to pq.  Explain why 

i)	$a^{p-1} = 1 \mod{p}$

ii)	$a^{q-1} = 1 \mod{q}$

iii)	$m = lcm(p-1, q-1)$ is $< (p-1)(q-1)$

iv)	$a^m = 1 \mod{(p-1)(q-1)}$

c)	Hence explain why numbers of the form n have no primitive roots.

d)	Show that all numbers of the form $n = pq$ (p and q both odd primes) in your range are included in set B.

In [ ]:
def isform_odd_pq(n):
    factors = [f for f in ifactors(n)]
    return (len(factors) == 2 and factors[0][1] == 1 and factors[0][0] % 2 == 1 
            and factors[1][1] == 1 and factors[1][1] == 1 and factors[1][0] % 2 == 1)

subset_n_pq = set(n for n in range(lower_range, upper_range + 1) if isform_odd_pq(n))
print(sorted(subset_n_pq))
print(all(pq in subset_b for pq in subset_n_pq))

__Question 4 (18 marks)__

Use the BabyStepsGiantSteps algorithm to find discrete logarithms xi of b mod n for the primitive root $a$ ($a^xi = b$) for each of the two examples assigned to you in the table below. Verify that your answer is correct by calculating ax mod m by hand using the method of modular exponentiation. 

| __Name__ | __b__ | __n__ | __a__ | __Method__        |
|----------|-------|-------|-------|-------------------|
| Stuart   | $37$  | $71$  | $33$  | BabyStepGiantStep |
| Stuart   | $29$  | $53$  | $14$  | BabyStepGiantStep |

The algorithm is based on a space–time tradeoff. It is a fairly simple modification of trial multiplication, the naive method of finding discrete logarithms.

Given a cyclic group $G$ of order $n$, a generator $\alpha$ of the group and a group element $\beta$, the problem is to find an integer $x$ such that:

&nbsp;&nbsp;&nbsp;&nbsp;$\alpha^x = \beta\,$.

In [ ]:
def baby_step_giant_step_dlp(a: int, b: int, n: int) -> int:
    """
    m ← Ceiling(√n)
    For all j where 0 ≤ j < m:
        Compute α_j and store the pair (j, α^j) in a table. (See § In practice)
    Compute α^{−m}.
    γ ← β. (set γ = β)
    For all i where 0 ≤ i < m:
        Check to see if γ is the second component (αj) of any pair in the table.
        If so, return im + j.
        If not, γ ← γ • α^−m.
    """
    m = math.ceil(math.sqrt(n))
    t = {pow(a, j, n): j for j in range(0, m) }
    a_m = modulo_inverse(pow(a, m, n), n)
    y = b
    x = 0

    for i in range(0, m):
        if y in t:
            x = (i * m + t[y]) % n
            break
        y = (y * a_m) % n

    assert x
    return x

In [ ]:
a = 33
b = 37
n = 71
x = baby_step_giant_step_dlp(a, b, n)
print(f"{x} solves b = a^x mod n: {b} = {a}^{x} (mod {n}) = {a ** x % n}"); 

a = 14
b = 29
n = 53
x = baby_step_giant_step_dlp(a, b, n)
print(f"{x} solves b = a^x mod n: {b} = {a}^{x} (mod {n}) = {a ** x % n}"); 

__Question 5 (18 marks)__


Use the Pohlig Helmann algorithm to find in the cyclic group of order n with the generating element __a__ for both the examples assigned to you below. Verify your answer in Maple.

| Name   | __b__  | __n__ | __a__  |  __Method__    |
|--------|--------|-------|--------|----------------|
| Stuart | $x^{97}$ | 343   | $x^{13}$ | Pohlig Hellman |
| Stuart | $x^{163}$| 3267  | $x^{19}$ | Pohlig Hellman |


In [ ]:
def pohlig_hellman(a:int, b: int, n: int) -> int:
    """ """
    return 0
# Adjusted parameters for demonstration
g = 5  # Adjusted generator
h = 3  # h = g^x in mod p, adjusted value
p = 47 # Adjusted prime number
factors = [(2, 1), (23, 1)]  # Prime factors of p - 1 with their multiplicities

# Call the Pohlig-Hellman algorithm again
x = pohlig_hellman(g, h, p, factors)
x


In [ ]:
n = 15525

In [ ]:
a = 13
b = 97
n = 343
x = pohlig_hellman(a, b, n)

b = 163
n = 3267
a = 19
x = pohlig_hellman(a, b, n)

__Question 6   (12 marks)__

Use the Pollard Rho method to verify your answer to the first example you were allocated in Question 4. 

In [ ]:
def pollard_rho(a: int, b: int, n: int, verbose=False) -> int:
    """Solve discrete logarithm using Pollard Rho algorithm"""

    def fgh(x_i: int, a_i: int, b_i: int):
        if (x_i % 3) == 0:
            return ((x_i * x_i) % n,  
                    (a_i * 2) % (n-1),  
                    (b_i * 2) % (n-1))
        if (x_i % 3) == 1:
            return ((x_i * a) % n, 
                    (a_i + 1) % (n - 1), 
                    b_i)
        # if (xi % 3) == 2:
        return ((x_i * b) % n, 
                a_i, 
                (b_i + 1) % (n - 1))

    xi, ai, bi = 1, 0, 0
    x2i, a2i, b2i = xi, 0, 0
    for i in range(1, n - 1):
        xi, ai, bi = fgh(xi, ai, bi)
        x2i, a2i, b2i = fgh(*fgh(x2i, a2i, b2i))
        if verbose:
            print(f"i={i}\txi={xi}\tai={ai}\tbi={bi}\tx2i={x2i}\ta2i={a2i}\tb2i={b2i}")
        if xi == x2i:
            break
            
    r = b2i - bi
    if r == 0:
        return None
    inv = modulo_inverse(r, n - 1)
    x = inv * (ai - a2i) % (n - 1)
    return x

n = 1019
a = 2 # generator
b = 5 # 2^10 = 5 (mod n)
x = pollard_rho(a, b, n, verbose=True)
print([a, b, n, x])
print(f"{a} ** {x} = {b} (mod {n}) => {a ** x % n} = {b} (mod {n})")


In [ ]:
a = 33
b = 37
n = 71
x = pollard_rho(a, b, n, verbose=True)
print([a, b, n, x])
print(f"x = {x} solves a^x mod m = b: {a}^{x} mod {n} = {a ** x % n} = {b}"); 
print(f"{a} ** {x} = {b} (mod {n}) => {a ** x % n} = {b} (mod {n})")

a = 14
b = 29
n = 53
x = pollard_rho(a, b, n, verbose=False)
print(f"x = {x} solves a^x mod m = b: {a}^{x} mod {n} = {a ** x % n} = {b}"); 
print(f"{a} ** {x} = {b} (mod {n}) => {a ** x % n} = {b} (mod {n})")

a = 2
b = 17
n = 53
x = pollard_rho(a, b, n, verbose=False)
print(f"x = {x} solves a^x mod m = b: {a}^{x} mod {n} = {a ** x % n} = {b}"); 
print(f"{a} ** {x} = {b} (mod {n}) => {a ** x % n} = {b} (mod {n})")

a = 7
b = 36
n = 53
x = pollard_rho(a, b, n, verbose=False)
print(f"x = {x} solves a^x mod m = b: {a}^{x} mod {n} = {a ** x % n} = {b}"); 
print(f"{a} ** {x} = {b} (mod {n}) => {a ** x % n} = {b} (mod {n})")


In [ ]:
import random

def pollard_rho_logarithm(alpha, beta, n):
    """
    Solve for x in the equation alpha^x = beta (mod n) using the Pollard Rho algorithm.
    """
    def f(x, A, B):
        if x % 3 == 0:
            return (beta * x) % n, (A + 1) % (n - 1), B
        elif x % 3 == 1:
            return (x * x) % n, (2 * A) % (n - 1), (2 * B) % (n - 1)
        else:
            return (alpha * x) % n, A, (B + 1) % (n - 1)

    x, A, B = 1, 0, 0
    X, AA, BB = x, A, B
    for count in range(1, n):
        x, A, B = f(x, A, B)
        X, AA, BB = f(*f(X, AA, BB))
        if x == X:
            break

    r = (B - BB) % (n - 1)
    if r == 0:
        return None

    r_inverse = pow(r, -1, n - 1)
    solution = (r_inverse * (AA - A)) % (n - 1)
    return solution

# Constants for the problem: solve alpha^x = beta (mod n)
alpha = 14
beta = 29
n = 53

# Use Pollard-Rho to find the discrete logarithm
solution = pollard_rho_logarithm(alpha, beta, n)
print(solution)
